# Import datasets

In [1]:
import numpy as np
import pandas as pd
def load_dataset(name):
    x = np.load(f'/datasets/nicolas_facchinetti/processed_data/{name}/x_trim_repeated_im.npy')
    y = np.load(f'/datasets/nicolas_facchinetti/processed_data/{name}/y_trim_repeated_im.npy')
    metadata = pd.read_csv(f'/datasets/nicolas_facchinetti/processed_data/{name}/trim_repeated_metadata.csv')
    return x, y, metadata

In [2]:
datasets = ["emodb", "emovo", "ravdess"]
data = {}
for d in datasets:
    x, y, md = load_dataset(d)
    data[d] = {}
    data[d]["x"] = x
    data[d]["y"] = y
    data[d]["metadata"] = md

In [3]:
data

{'emodb': {'x': array([[[[0.        , 0.        , 0.00257353],
           [0.        , 0.        , 0.00882353],
           [0.        , 0.        , 0.00882353],
           ...,
           [0.35588235, 0.1       , 0.35588235],
           [0.5382353 , 0.16176471, 0.32352942],
           [0.52003676, 0.15514706, 0.31194854]],
  
          [[0.        , 0.        , 0.00514706],
           [0.        , 0.        , 0.01568628],
           [0.        , 0.        , 0.01568628],
           ...,
           [0.5058824 , 0.14509805, 0.5058824 ],
           [0.7607843 , 0.23137255, 0.45882353],
           [0.73651963, 0.22254902, 0.44338235]],
  
          [[0.        , 0.        , 0.00514706],
           [0.        , 0.        , 0.01568628],
           [0.        , 0.        , 0.01568628],
           ...,
           [0.49117646, 0.1392157 , 0.5088235 ],
           [0.85490197, 0.28431374, 0.41470587],
           [0.82732844, 0.27383578, 0.4009191 ]],
  
          ...,
  
          [[0.        , 0.

# Define architectures

In [4]:
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.models import Sequential 
from tensorflow.keras.callbacks import EarlyStopping 
from tensorflow.keras.layers import Conv2D, Flatten, Dense, Dropout, MaxPooling2D, LSTM, TimeDistributed, InputLayer, Reshape, BatchNormalization

2022-11-08 18:00:34.625378: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


Search the best architecture

In [5]:
archs = []

In [6]:
arch0 = Sequential([
    InputLayer(input_shape=(256,256,3)),
    Reshape((16,16,256,3)),
    BatchNormalization(),
    TimeDistributed(Conv2D(16, kernel_size=(3,3), activation='relu', strides=(1))),
    TimeDistributed(MaxPooling2D(pool_size=(2))),
    TimeDistributed(Conv2D(32, kernel_size=(3,3), activation='relu', strides=(1))),
    TimeDistributed(MaxPooling2D(pool_size=(2,2), strides=(2))),

    TimeDistributed(Flatten()),
    LSTM(128, return_sequences=False),
    Dropout(0.2),
    Flatten(),
    Dense(7, activation='softmax')
])
arch0.summary()

archs.append(arch0.get_config())

2022-11-08 18:00:35.554089: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-11-08 18:00:35.555209: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-11-08 18:00:35.662219: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:15:00.0 name: NVIDIA TITAN V computeCapability: 7.0
coreClock: 1.455GHz coreCount: 80 deviceMemorySize: 11.78GiB deviceMemoryBandwidth: 607.97GiB/s
2022-11-08 18:00:35.662260: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-11-08 18:00:35.664035: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2022-11-08 18:00:35.664116: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.10
2022-1

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape (Reshape)            (None, 16, 16, 256, 3)    0         
_________________________________________________________________
batch_normalization (BatchNo (None, 16, 16, 256, 3)    12        
_________________________________________________________________
time_distributed (TimeDistri (None, 16, 14, 254, 16)   448       
_________________________________________________________________
time_distributed_1 (TimeDist (None, 16, 7, 127, 16)    0         
_________________________________________________________________
time_distributed_2 (TimeDist (None, 16, 5, 125, 32)    4640      
_________________________________________________________________
time_distributed_3 (TimeDist (None, 16, 2, 62, 32)     0         
_________________________________________________________________
time_distributed_4 (TimeDist (None, 16, 3968)          0

In [7]:
arch1 = Sequential([
    InputLayer(input_shape=(256,256,3)),
    Reshape((16,16,256,3)),
    BatchNormalization(),
    TimeDistributed(Conv2D(32, kernel_size=(3,3), activation='relu', strides=(1))),
    TimeDistributed(MaxPooling2D(pool_size=(2,2))),
    TimeDistributed(Conv2D(64, kernel_size=(3,3), activation='relu', strides=(1))),
    TimeDistributed(MaxPooling2D(pool_size=(2,2), strides=(2))),

    TimeDistributed(Flatten()),
    LSTM(128, return_sequences=False),
    Dropout(0.2),
    Flatten(),
    Dense(7, activation='softmax')
])
arch1.summary()

archs.append(arch1.get_config())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_1 (Reshape)          (None, 16, 16, 256, 3)    0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 16, 16, 256, 3)    12        
_________________________________________________________________
time_distributed_5 (TimeDist (None, 16, 14, 254, 32)   896       
_________________________________________________________________
time_distributed_6 (TimeDist (None, 16, 7, 127, 32)    0         
_________________________________________________________________
time_distributed_7 (TimeDist (None, 16, 5, 125, 64)    18496     
_________________________________________________________________
time_distributed_8 (TimeDist (None, 16, 2, 62, 64)     0         
_________________________________________________________________
time_distributed_9 (TimeDist (None, 16, 7936)         

In [8]:
arch2 = Sequential([
    InputLayer(input_shape=(256,256,3)),
    Reshape((16,16,256,3)),
    BatchNormalization(),
    TimeDistributed(Conv2D(32, kernel_size=(3,3), activation='relu', strides=(1))),
    TimeDistributed(MaxPooling2D(pool_size=(2))),
    TimeDistributed(Conv2D(64, kernel_size=(3,3), activation='relu', strides=(1))),
    TimeDistributed(MaxPooling2D(pool_size=(2,2), strides=(2))),

    TimeDistributed(Flatten()),
    LSTM(256, return_sequences=False),
    Dropout(0.2),
    Flatten(),
    Dense(7, activation='softmax')
])
arch2.summary()

archs.append(arch2.get_config())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_2 (Reshape)          (None, 16, 16, 256, 3)    0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 16, 16, 256, 3)    12        
_________________________________________________________________
time_distributed_10 (TimeDis (None, 16, 14, 254, 32)   896       
_________________________________________________________________
time_distributed_11 (TimeDis (None, 16, 7, 127, 32)    0         
_________________________________________________________________
time_distributed_12 (TimeDis (None, 16, 5, 125, 64)    18496     
_________________________________________________________________
time_distributed_13 (TimeDis (None, 16, 2, 62, 64)     0         
_________________________________________________________________
time_distributed_14 (TimeDis (None, 16, 7936)         

In [9]:
arch3 = Sequential([
    InputLayer(input_shape=(256,256,3)),
    Reshape((8,32,256,3)),
    BatchNormalization(),
    TimeDistributed(Conv2D(16, kernel_size=(3,3), activation='relu', strides=(1))),
    TimeDistributed(MaxPooling2D(pool_size=(2,2))),
    TimeDistributed(Conv2D(32, kernel_size=(3,3), activation='relu', strides=(1))),
    TimeDistributed(MaxPooling2D(pool_size=(2,2), strides=(2))),
    TimeDistributed(Conv2D(64, kernel_size=(3,3), activation='relu', strides=(1))),
    TimeDistributed(MaxPooling2D(pool_size=(2,2), strides=(2))),

    TimeDistributed(Flatten()),
    LSTM(128, return_sequences=False),
    Dropout(0.2),
    Flatten(),
    Dense(7, activation='softmax')
])
arch1.summary()

archs.append(arch1.get_config())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_1 (Reshape)          (None, 16, 16, 256, 3)    0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 16, 16, 256, 3)    12        
_________________________________________________________________
time_distributed_5 (TimeDist (None, 16, 14, 254, 32)   896       
_________________________________________________________________
time_distributed_6 (TimeDist (None, 16, 7, 127, 32)    0         
_________________________________________________________________
time_distributed_7 (TimeDist (None, 16, 5, 125, 64)    18496     
_________________________________________________________________
time_distributed_8 (TimeDist (None, 16, 2, 62, 64)     0         
_________________________________________________________________
time_distributed_9 (TimeDist (None, 16, 7936)         

In [10]:
arch4 = Sequential([
    InputLayer(input_shape=(256,256,3)),
    Reshape((8,32,256,3)),
    BatchNormalization(),
    TimeDistributed(Conv2D(32, kernel_size=(3,3), activation='relu', strides=(1))),
    TimeDistributed(MaxPooling2D(pool_size=(2,2))),
    TimeDistributed(Conv2D(64, kernel_size=(3,3), activation='relu', strides=(1))),
    TimeDistributed(MaxPooling2D(pool_size=(2,2), strides=(2))),
    TimeDistributed(Conv2D(128, kernel_size=(3,3), activation='relu', strides=(1))),
    TimeDistributed(MaxPooling2D(pool_size=(2,2), strides=(2))),

    TimeDistributed(Flatten()),
    LSTM(128, return_sequences=False),
    Dropout(0.2),
    Flatten(),
    Dense(7, activation='softmax')
])
arch4.summary()

archs.append(arch4.get_config())

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_4 (Reshape)          (None, 8, 32, 256, 3)     0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 8, 32, 256, 3)     12        
_________________________________________________________________
time_distributed_22 (TimeDis (None, 8, 30, 254, 32)    896       
_________________________________________________________________
time_distributed_23 (TimeDis (None, 8, 15, 127, 32)    0         
_________________________________________________________________
time_distributed_24 (TimeDis (None, 8, 13, 125, 64)    18496     
_________________________________________________________________
time_distributed_25 (TimeDis (None, 8, 6, 62, 64)      0         
_________________________________________________________________
time_distributed_26 (TimeDis (None, 8, 4, 60, 128)    

In [11]:
arch5 = Sequential([
    InputLayer(input_shape=(256,256,3)),
    Reshape((8,32,256,3)),
    BatchNormalization(),
    TimeDistributed(Conv2D(32, kernel_size=(3,3), activation='relu', strides=(1))),
    TimeDistributed(MaxPooling2D(pool_size=(2,2))),
    TimeDistributed(Conv2D(64, kernel_size=(3,3), activation='relu', strides=(1))),
    TimeDistributed(MaxPooling2D(pool_size=(2,2), strides=(2))),
    TimeDistributed(Conv2D(128, kernel_size=(3,3), activation='relu', strides=(1))),
    TimeDistributed(MaxPooling2D(pool_size=(2,2), strides=(2))),

    TimeDistributed(Flatten()),
    LSTM(256, return_sequences=False),
    Dropout(0.2),
    Flatten(),
    Dense(7, activation='softmax')
])
arch5.summary()

archs.append(arch5.get_config())

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_5 (Reshape)          (None, 8, 32, 256, 3)     0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 8, 32, 256, 3)     12        
_________________________________________________________________
time_distributed_29 (TimeDis (None, 8, 30, 254, 32)    896       
_________________________________________________________________
time_distributed_30 (TimeDis (None, 8, 15, 127, 32)    0         
_________________________________________________________________
time_distributed_31 (TimeDis (None, 8, 13, 125, 64)    18496     
_________________________________________________________________
time_distributed_32 (TimeDis (None, 8, 6, 62, 64)      0         
_________________________________________________________________
time_distributed_33 (TimeDis (None, 8, 4, 60, 128)    

In [12]:
archs

[{'name': 'sequential',
  'layers': [{'class_name': 'InputLayer',
    'config': {'batch_input_shape': (None, 256, 256, 3),
     'dtype': 'float32',
     'sparse': False,
     'ragged': False,
     'name': 'input_1'}},
   {'class_name': 'Reshape',
    'config': {'name': 'reshape',
     'trainable': True,
     'dtype': 'float32',
     'target_shape': (16, 16, 256, 3)}},
   {'class_name': 'BatchNormalization',
    'config': {'name': 'batch_normalization',
     'trainable': True,
     'dtype': 'float32',
     'axis': ListWrapper([4]),
     'momentum': 0.99,
     'epsilon': 0.001,
     'center': True,
     'scale': True,
     'beta_initializer': {'class_name': 'Zeros', 'config': {}},
     'gamma_initializer': {'class_name': 'Ones', 'config': {}},
     'moving_mean_initializer': {'class_name': 'Zeros', 'config': {}},
     'moving_variance_initializer': {'class_name': 'Ones', 'config': {}},
     'beta_regularizer': None,
     'gamma_regularizer': None,
     'beta_constraint': None,
     'gamm

# Search best architecturs for all datasets

In [13]:
def create_model(config):
    new_model = keras.Sequential.from_config(config)
    new_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return new_model

In [14]:
def leave_one_speaker_out(data, n=5):
    speakers = data.actor.unique()
    # random select n actors as test
    selected = np.random.choice(speakers, size=n, replace=False)
    
    myCViterator = []
    for i in selected:
        trainIndices = data[data['actor']!=i ].index.values.astype(int)
        testIndices =  data[data['actor']==i ].index.values.astype(int)
        myCViterator.append((trainIndices, testIndices))
    return myCViterator

In [15]:
from sklearn.model_selection import cross_val_score
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

results_arch = {}
for count, a in enumerate(archs):
    print(f'Architecture {count} of {len(archs)-1}')
    
    def create_model():
        new_model = tensorflow.keras.Sequential.from_config(a)
        new_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        return new_model
    
    model = KerasClassifier(build_fn=create_model, epochs=30, batch_size=32, verbose=0)
          
    results_arch[f"arch{count}"] = {}

    for d in data:
        print(f"\tTraining on {d}")
        x = data[d]["x"]
        y = data[d]["y"]
        metadata = data[d]["metadata"]
        res = cross_val_score(model, x, y, cv=leave_one_speaker_out(metadata, n=3))
        results_arch[f"arch{count}"][d] = res

Architecture 0 of 5
	Training on emodb


2022-11-08 18:00:40.703896: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-11-08 18:00:40.719845: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3301490000 Hz
2022-11-08 18:00:41.720927: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2022-11-08 18:00:41.913019: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7


	Training on emovo
	Training on ravdess
Architecture 1 of 5
	Training on emodb
	Training on emovo
	Training on ravdess
Architecture 2 of 5
	Training on emodb
	Training on emovo
	Training on ravdess
Architecture 3 of 5
	Training on emodb
	Training on emovo


	Training on ravdess
Architecture 4 of 5
	Training on emodb
	Training on emovo
	Training on ravdess
Architecture 5 of 5
	Training on emodb
	Training on emovo
	Training on ravdess


In [16]:
results_arch

{'arch0': {'emodb': array([0.71875   , 0.47674417, 0.41269842]),
  'emovo': array([0.25954199, 0.33333334, 0.34640524]),
  'ravdess': array([0.48214287, 0.55263156, 0.54545456])},
 'arch1': {'emodb': array([0.51162791, 0.5272727 , 0.62962961]),
  'emovo': array([0.20454545, 0.30718955, 0.22900763]),
  'ravdess': array([0.53947371, 0.43859649, 0.30645162])},
 'arch2': {'emodb': array([0.66666669, 0.50666666, 0.57746476]),
  'emovo': array([0.27044025, 0.19083969, 0.18939394]),
  'ravdess': array([0.54098362, 0.46969697, 0.42857143])},
 'arch3': {'emodb': array([0.54651165, 0.47999999, 0.69387758]),
  'emovo': array([0.30718955, 0.21383648, 0.31944445]),
  'ravdess': array([0.45901638, 0.4107143 , 0.42105263])},
 'arch4': {'emodb': array([0.62962961, 0.52325583, 0.64583331]),
  'emovo': array([0.21374045, 0.20454545, 0.30718955]),
  'ravdess': array([0.42622951, 0.50724638, 0.53333336])},
 'arch5': {'emodb': array([0.59154928, 0.6122449 , 0.51162791]),
  'emovo': array([0.3263889 , 0.270

In [17]:
for arch in results_arch:
    print(arch)
    for d in results_arch[arch]:
        mean = round(np.mean(results_arch[arch][d])*100, 3)
        std = round(np.std(results_arch[arch][d])*100, 3)
        print(f'\t{d}: mean {mean}, std dev {std}')

arch0
	emodb: mean 53.606, std dev 13.18
	emovo: mean 31.309, std dev 3.824
	ravdess: mean 52.674, std dev 3.167
arch1
	emodb: mean 55.618, std dev 5.233
	emovo: mean 24.691, std dev 4.378
	ravdess: mean 42.817, std dev 9.542
arch2
	emodb: mean 58.36, std dev 6.546
	emovo: mean 21.689, std dev 3.787
	ravdess: mean 47.975, std dev 4.644
arch3
	emodb: mean 57.346, std dev 8.937
	emovo: mean 28.016, std dev 4.716
	ravdess: mean 43.026, std dev 2.077
arch4
	emodb: mean 59.957, std dev 5.437
	emovo: mean 24.183, std dev 4.637
	ravdess: mean 48.894, std dev 4.56
arch5
	emodb: mean 57.181, std dev 4.338
	emovo: mean 27.975, std dev 3.49
	ravdess: mean 46.687, std dev 4.222


# Hyperparameter optimization

In [142]:
chosen = archs[4]
def create_model_opt():
    new_model = tensorflow.keras.Sequential.from_config(chosen)
    new_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return new_model
    
model = KerasClassifier(build_fn=create_model_opt, verbose=0)

In [143]:
from sklearn.model_selection import GridSearchCV
# define the grid search parameters
batch_size = [16, 32, 64, 128]
epochs = [30, 50, 70]
param_grid = dict(batch_size=batch_size, epochs=epochs)

In [145]:
results_opt = {}
for d in data:
    print(f'Dataset {d}')
    
    x = data[d]["x"]
    y = data[d]["y"]
    metadata = data[d]["metadata"]
    cv = leave_one_speaker_out(metadata, n=3)
    
    grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=cv, verbose=3)
    grid_result = grid.fit(x, y)
    results_opt[d] = grid_result
    

Dataset emodb
Fitting 3 folds for each of 12 candidates, totalling 36 fits
[CV 1/3] END ..........batch_size=16, epochs=30;, score=0.649 total time=  18.0s
[CV 2/3] END ..........batch_size=16, epochs=30;, score=0.667 total time=  19.4s
[CV 3/3] END ..........batch_size=16, epochs=30;, score=0.631 total time=  18.5s
[CV 1/3] END ..........batch_size=16, epochs=50;, score=0.662 total time=  28.2s
[CV 2/3] END ..........batch_size=16, epochs=50;, score=0.564 total time=  30.4s
[CV 3/3] END ..........batch_size=16, epochs=50;, score=0.600 total time=  29.8s
[CV 1/3] END ..........batch_size=16, epochs=70;, score=0.662 total time=  40.0s
[CV 2/3] END ..........batch_size=16, epochs=70;, score=0.615 total time=  42.0s
[CV 3/3] END ..........batch_size=16, epochs=70;, score=0.585 total time=  40.4s
[CV 1/3] END ..........batch_size=32, epochs=30;, score=0.714 total time=  15.6s
[CV 2/3] END ..........batch_size=32, epochs=30;, score=0.641 total time=  16.6s
[CV 3/3] END ..........batch_size=

[CV 2/3] END ..........batch_size=64, epochs=30;, score=0.615 total time=  18.1s
[CV 3/3] END ..........batch_size=64, epochs=30;, score=0.600 total time=  16.2s
[CV 1/3] END ..........batch_size=64, epochs=50;, score=0.636 total time=  25.2s
[CV 2/3] END ..........batch_size=64, epochs=50;, score=0.641 total time=  26.6s
[CV 3/3] END ..........batch_size=64, epochs=50;, score=0.600 total time=  25.6s
[CV 1/3] END ..........batch_size=64, epochs=70;, score=0.649 total time=  34.8s
[CV 2/3] END ..........batch_size=64, epochs=70;, score=0.615 total time=  36.5s
[CV 3/3] END ..........batch_size=64, epochs=70;, score=0.662 total time=  35.3s
[CV 1/3] END .........batch_size=128, epochs=30;, score=0.623 total time=  16.4s
[CV 2/3] END .........batch_size=128, epochs=30;, score=0.667 total time=  15.4s
[CV 3/3] END .........batch_size=128, epochs=30;, score=0.554 total time=  15.0s


[CV 1/3] END .........batch_size=128, epochs=50;, score=0.662 total time=  23.3s
[CV 2/3] END .........batch_size=128, epochs=50;, score=0.641 total time=  24.7s
[CV 3/3] END .........batch_size=128, epochs=50;, score=0.600 total time=  23.5s
[CV 1/3] END .........batch_size=128, epochs=70;, score=0.649 total time=  31.8s
[CV 2/3] END .........batch_size=128, epochs=70;, score=0.692 total time=  35.1s
[CV 3/3] END .........batch_size=128, epochs=70;, score=0.662 total time=  32.3s
Dataset emovo
Fitting 3 folds for each of 12 candidates, totalling 36 fits
[CV 1/3] END ..........batch_size=16, epochs=30;, score=0.227 total time=  21.2s
[CV 2/3] END ..........batch_size=16, epochs=30;, score=0.311 total time=  21.1s
[CV 3/3] END ..........batch_size=16, epochs=30;, score=0.250 total time=  20.7s
[CV 1/3] END ..........batch_size=16, epochs=50;, score=0.286 total time=  33.9s
[CV 2/3] END ..........batch_size=16, epochs=50;, score=0.361 total time=  33.5s
[CV 3/3] END ..........batch_size=

[CV 1/3] END ..........batch_size=64, epochs=50;, score=0.252 total time=  29.1s
[CV 2/3] END ..........batch_size=64, epochs=50;, score=0.352 total time=  29.3s
[CV 3/3] END ..........batch_size=64, epochs=50;, score=0.258 total time=  28.6s
[CV 1/3] END ..........batch_size=64, epochs=70;, score=0.252 total time=  40.7s
[CV 2/3] END ..........batch_size=64, epochs=70;, score=0.369 total time=  40.7s
[CV 3/3] END ..........batch_size=64, epochs=70;, score=0.266 total time=  41.4s
[CV 1/3] END .........batch_size=128, epochs=30;, score=0.328 total time=  17.6s
[CV 2/3] END .........batch_size=128, epochs=30;, score=0.361 total time=  17.6s
[CV 3/3] END .........batch_size=128, epochs=30;, score=0.305 total time=  17.3s
[CV 1/3] END .........batch_size=128, epochs=50;, score=0.252 total time=  27.2s
[CV 2/3] END .........batch_size=128, epochs=50;, score=0.320 total time=  26.9s


[CV 3/3] END .........batch_size=128, epochs=50;, score=0.305 total time=  26.6s
[CV 1/3] END .........batch_size=128, epochs=70;, score=0.269 total time=  37.2s
[CV 2/3] END .........batch_size=128, epochs=70;, score=0.311 total time=  38.2s
[CV 3/3] END .........batch_size=128, epochs=70;, score=0.273 total time=  36.7s
Dataset ravdess
Fitting 3 folds for each of 12 candidates, totalling 36 fits
[CV 1/3] END ..........batch_size=16, epochs=30;, score=0.316 total time=  46.9s
[CV 2/3] END ..........batch_size=16, epochs=30;, score=0.433 total time=  52.8s
[CV 3/3] END ..........batch_size=16, epochs=30;, score=0.486 total time=  52.4s
[CV 1/3] END ..........batch_size=16, epochs=50;, score=0.354 total time= 1.3min
[CV 2/3] END ..........batch_size=16, epochs=50;, score=0.500 total time= 1.5min
[CV 3/3] END ..........batch_size=16, epochs=50;, score=0.403 total time= 1.4min
[CV 1/3] END ..........batch_size=16, epochs=70;, score=0.329 total time= 1.8min
[CV 2/3] END ..........batch_siz

[CV 1/3] END ..........batch_size=32, epochs=70;, score=0.329 total time= 1.7min
[CV 2/3] END ..........batch_size=32, epochs=70;, score=0.433 total time= 1.7min
[CV 3/3] END ..........batch_size=32, epochs=70;, score=0.444 total time= 1.7min
[CV 1/3] END ..........batch_size=64, epochs=30;, score=0.392 total time=  45.8s
[CV 2/3] END ..........batch_size=64, epochs=30;, score=0.483 total time=  47.9s
[CV 3/3] END ..........batch_size=64, epochs=30;, score=0.514 total time=  45.8s
[CV 1/3] END ..........batch_size=64, epochs=50;, score=0.291 total time= 1.2min
[CV 2/3] END ..........batch_size=64, epochs=50;, score=0.450 total time= 1.3min
[CV 3/3] END ..........batch_size=64, epochs=50;, score=0.514 total time= 1.2min
[CV 1/3] END ..........batch_size=64, epochs=70;, score=0.342 total time= 1.7min


[CV 2/3] END ..........batch_size=64, epochs=70;, score=0.533 total time= 1.7min
[CV 3/3] END ..........batch_size=64, epochs=70;, score=0.458 total time= 1.8min
[CV 1/3] END .........batch_size=128, epochs=30;, score=0.380 total time=  42.4s
[CV 2/3] END .........batch_size=128, epochs=30;, score=0.533 total time=  42.4s
[CV 3/3] END .........batch_size=128, epochs=30;, score=0.514 total time=  42.3s
[CV 1/3] END .........batch_size=128, epochs=50;, score=0.468 total time= 1.1min
[CV 2/3] END .........batch_size=128, epochs=50;, score=0.450 total time= 1.1min
[CV 3/3] END .........batch_size=128, epochs=50;, score=0.472 total time= 1.1min
[CV 1/3] END .........batch_size=128, epochs=70;, score=0.430 total time= 1.6min
[CV 2/3] END .........batch_size=128, epochs=70;, score=0.450 total time= 1.6min
[CV 3/3] END .........batch_size=128, epochs=70;, score=0.514 total time= 1.6min


In [146]:
# summarize results
for d in results_opt:
    print(d)
    grid_result = results_opt[d]
    print("\tBest: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
    means = grid_result.cv_results_['mean_test_score']
    stds = grid_result.cv_results_['std_test_score']
    params = grid_result.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
        print("\t%f (%f) with: %r" % (mean, stdev, param))

emodb
	Best: 0.677411 using {'batch_size': 32, 'epochs': 30}
	0.648929 (0.014658) with: {'batch_size': 16, 'epochs': 30}
	0.608813 (0.040586) with: {'batch_size': 16, 'epochs': 50}
	0.620779 (0.031958) with: {'batch_size': 16, 'epochs': 70}
	0.677411 (0.029910) with: {'batch_size': 32, 'epochs': 30}
	0.666245 (0.028795) with: {'batch_size': 32, 'epochs': 50}
	0.654168 (0.015106) with: {'batch_size': 32, 'epochs': 70}
	0.638894 (0.044571) with: {'batch_size': 64, 'epochs': 30}
	0.625796 (0.018340) with: {'batch_size': 64, 'epochs': 50}
	0.642091 (0.019529) with: {'batch_size': 64, 'epochs': 70}
	0.614630 (0.046472) with: {'batch_size': 128, 'epochs': 30}
	0.634454 (0.025870) with: {'batch_size': 128, 'epochs': 50}
	0.667732 (0.018076) with: {'batch_size': 128, 'epochs': 70}
emovo
	Best: 0.331025 using {'batch_size': 128, 'epochs': 30}
	0.262789 (0.035696) with: {'batch_size': 16, 'epochs': 30}
	0.309207 (0.036426) with: {'batch_size': 16, 'epochs': 50}
	0.257669 (0.008121) with: {'batch